# Word Embedding
https://www.youtube.com/watch?v=5PL0TmQhItY

https://towardsdatascience.com/implementing-word2vec-in-pytorch-skip-gram-model-e6bae040d2fb

http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

In [42]:
!pip install torch nltk pandas seaborn numpy

In [43]:
import nltk
import torch
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
file_name = "gdrive/MyDrive/tweets.csv"

In [ ]:
from nltk.stem import WordNetLemmatizer
stopwords = nltk.corpus.stopwords.words('english')
  
lemmatizer = WordNetLemmatizer()

In [ ]:
tweets_df = pd.read_csv(file_name,  header=None, encoding='ISO-8859-1', names=["target", "ids", "date", "flag", "user", "text"])

In [ ]:
tweets_df.head()

In [ ]:
sentences = [nltk.sent_tokenize(tweet) for tweet in tweets_df["text"].iloc[:10000]]
sentences = list(itertools.chain(*sentences))

def cleanup_text(sentences):
  clean_sentences = []
  for sentence in sentences:
    clean_sentence = []
    for token in sentence.split(" "):
      if token in stopwords or not token.isalpha():
        continue
      clean_sentence.append(lemmatizer.lemmatize(token))
    clean_sentences.append(clean_sentence)
  return clean_sentences


# simplify and clean sentences
sentences = cleanup_text(sentences)

In [ ]:
vocabulary = []
for sentence in sentences:
  for token in sentence:
    if token not in vocabulary:
      vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [ ]:
window_size = 2

ngrams = [nltk.ngrams(sentence, window_size) for sentence in sentences if len(sentence) > window_size]
ngrams = list(itertools.chain(*ngrams))

def get_ngrams_idx(ngrams):
  """
  Function to map ngram of variable length to idx
  """
  ngrams_idx = []
  for ngram in ngrams:
    word_idx = []
    for word in ngram:
      word_idx.append(word2idx[word])
    ngrams_idx.append(tuple(word_idx))
  return ngrams_idx


idx_pairs = get_ngrams_idx(ngrams)

# Word2Vec implementation

## Skip-n-gram

In [ ]:
from torch import nn

def get_input_layer(word_id):
  x = torch.zeros(vocabulary_size)
  x[word_id] = 1
  return x

class WordEmbedder(nn.Sequential):

    def __init__(self, vocabulary_size, embedding_dims):
        super(WordEmbedder, self).__init__()
        self.embedding_dims = embedding_dims
        self.vocabulary_size = vocabulary_size
        self.layer = nn.Embedding(self.vocabulary_size,self.embedding_dims, sparse=True)
        self.activation = nn.Softmax(self.vocabulary_size)

    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

model = WordEmbedder(vocabulary_size, 5)

# Matrix Decomposition

# Glove

# Keras embedding layer